In [ ]:
!git clone --single-branch --branch paper https://github.com/WongKinYiu/yolor
%cd yolor

In [ ]:
# Install necessary dependencies
!pip install -qr requirements.txt

In [ ]:
# Install Mish CUDA
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!git reset --hard 6f38976064cbcc4782f4212d7c0c5f6dd5e315a8
!python setup.py build install
%cd ..

In [ ]:
# Install PyTorch Wavelets
!git clone https://github.com/fbcotter/pytorch_wavelets
%cd pytorch_wavelets
!pip install .
%cd ..

In [ ]:
# Dataset
!curl -L "https://app.roboflow.com/ds/XXXXX" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Modification de data.yaml pour fonctionner sur colab
import os
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolor/data.yaml

train: /content/yolor/train/images
val: /content/yolor/valid/images

nc: 1
names: ['person']

In [ ]:
#from distutils.dir_util import copy_tree
#copy_tree("/content/yolor/train/labels", "/content/yolor/train/images")
#copy_tree("/content/yolor/valid/labels", "/content/yolor/valid/images")

In [ ]:
import os
 
os.chdir('/content/yolor/train/images')
 
for count, f in enumerate(os.listdir()):
    f_name, f_ext = os.path.splitext(f)
    f_name = f_name[:len(f_name)-40]
 
    new_name = f'{f_name}{f_ext}'
    os.rename(f, new_name)

In [ ]:
import os
 
os.chdir('/content/yolor/train/labels')
 
for count, f in enumerate(os.listdir()):
    f_name, f_ext = os.path.splitext(f)
    f_name = f_name[:len(f_name)-40]
 
    new_name = f'{f_name}{f_ext}'
    os.rename(f, new_name)

In [ ]:
import os
 
os.chdir('/content/yolor/valid/images')
 
for count, f in enumerate(os.listdir()):
    f_name, f_ext = os.path.splitext(f)
    f_name = f_name[:len(f_name)-40]
 
    new_name = f'{f_name}{f_ext}'
    os.rename(f, new_name)

In [ ]:
import os
 
os.chdir('/content/yolor/valid/labels')
 
for count, f in enumerate(os.listdir()):
    f_name, f_ext = os.path.splitext(f)
    f_name = f_name[:len(f_name)-40]
 
    new_name = f'{f_name}{f_ext}'
    os.rename(f, new_name)

In [ ]:
# Poids Paper
%cd /content/yolor
!gdown 1WyzcN1-I0n8BoeRhi_xVt8C5msqdx_7k

In [ ]:
%pip install -q wandb
import wandb
wandb.login()

In [ ]:
# train
%cd /content/yolor
!python train.py --batch-size 16 --img 640 640 --data '/content/yolor/data.yaml' --weights '/content/yolor/yolor-p6.pt' --device 0 --name yolor-p6 --hyp '/content/yolor/data/hyp.scratch.1280.yaml' --epochs 10

In [ ]:
# test
!python detect.py --weights "/content/yolor/runs/train/yolor-p6/weights/best.pt" --conf 0.25 --source /content/yolor/valid/images --img-size 640

In [ ]:
!pip install onnx
!pip install torch==1.9
!pip install onnx2keras

In [ ]:
# Export
!python models/export.py --weights "/content/yolor/runs/train/yolor-p6/weights/best.pt" --img-size 640 640